In [1]:
import pickle
import os.path
from whoosh.index import create_in
from whoosh.fields import *
from whoosh.qparser import QueryParser
from whoosh import qparser
import random
from nltk.corpus import stopwords
import nltk
# import pyttsx3
# import speech_recognition as sr
import glob
# import pydub
import asr.align
# from gtts import gTTS
import os
import soundfile as sf
from python_speech_features import mfcc
from python_speech_features import fbank
# nltk.download('stopwords')
import lshknn
import numpy as np
import fnmatch
import json
import sys
sys.path.append("/home/ubuntu/python-KNN/")
import kdtree

In [2]:
transDict = pickle.load(open('transDict2', 'r'))
convertedDict = pickle.load(open('speechRecConvertDict2', 'r'))
queryList = pickle.load(open('queryList2', 'r'))
queryConvertedDict = pickle.load(open('queryConvertedDict2', 'r'))

In [3]:
# encodeForQuery = []
# encodeQueryDict = dict()
# count = 0
# for root, dirnames, filenames in os.walk('/home/ubuntu/autoencoder_encoding/'):
#     for filename in fnmatch.filter(filenames, '*.wav.json'):
#         curr = np.array(json.load(open(root+filename, 'r')))
# #         print(curr.shape)
#         for c in curr:
#             encodeForQuery.append(c)
#             encodeQueryDict[count] = filename
#             count += 1
# #             print(len(c))
# #         print(len(curr))
# #         print(len(curr[0]))
#     print(count)
#     print(len(encodeForQuery))
    
# encodeForAudio = []
# encodeAudioDict = dict()
# count = 0
# for root, dirnames, filenames in os.walk('/home/ubuntu/autoencoder_encoding/'):
#     for filename in fnmatch.filter(filenames, '*.flac.json'):
#         curr = json.load(open(root+filename, 'r'))
#         for c in curr:
#             encodeForAudio.append(c)
#             encodeAudioDict[count] = filename
#             count += 1
#             if(len(c)!=120):
#                 print(len(c))
# #         print(len(curr))
# #         print(len(curr[0]))
#     print(count)
#     print(len(encodeForAudio))

In [4]:
minQueryLen = 100000
featSize = 30
stride = 1
# mfccDict = dict(list())
audioMfcc = []
audioPosnName = dict()
count = 0
j = 0
for root, dirnames, filenames in os.walk('/home/ubuntu/DevData-AudioBooks/dev-clean/'):
    for filename in fnmatch.filter(filenames, '*.wav'):
        count += 1
        with open(os.path.join(root, filename), 'rb') as f:
            data, samplerate = sf.read(f)
#         print("Read sounfile done")
        mfcc_feat = np.array(mfcc(data, samplerate))
#         print("Mfcc done")
        mfcclen = len(mfcc_feat)
        i = 0
        while(i+featSize<mfcclen):
            audioMfcc.append(mfcc_feat[i:i+featSize].flatten())
#         if(len(mfcc_feat)<minQueryLen):
#             minQueryLen = len(mfcc_feat)
            audioPosnName[j] = filename.split('.')[0]
            i += stride
            j += 1
print(len(audioMfcc))
print(mfcclen)
print(count)

1856254
613
2705


In [95]:
# mfccNumpyArray = np.array()

In [96]:
# minQueryLen = 100000
# count = 0
# for filename in glob.glob('/home/ubuntu/spokenkeywordsearch/GroundTruth/**/*.wav'):
#     count += 1
#     with open(filename, 'rb') as f:
#         data, samplerate = sf.read(f)
# #     print(len(data))
# #     print(samplerate)
#     mfcc_feat = mfcc(data, samplerate, nfft=600)
#     if(len(mfcc_feat)<minQueryLen):
#         minQueryLen = len(mfcc_feat)
# #     print(len(mfcc_feat))
# print(minQueryLen)
# print(count)

In [5]:
minQueryLen = 100000
maxQueryLen = 0
mfccDict = dict(list())
mfccNumpyArray = []
mfccPosnName = dict()
count = 0
j = 0
for root, dirnames, filenames in os.walk('/home/ubuntu/spokenkeywordsearch/GroundTruth/'):
    for filename in fnmatch.filter(filenames, '*.wav'):
        count += 1
        with open(os.path.join(root, filename), 'rb') as f:
            data, samplerate = sf.read(f)
        mfcc_feat = np.array(mfcc(data, samplerate, nfft=600))
        mfcclen = len(mfcc_feat)
        if(minQueryLen>mfcclen):
            minQueryLen = mfcclen
            minId = f
        if(maxQueryLen<mfcclen):
            maxQueryLen = mfcclen
            maxId = f
        i = 0
        while(i+featSize<mfcclen):
            mfccNumpyArray.append(mfcc_feat[i:i+featSize].flatten())
#         if(len(mfcc_feat)<minQueryLen):
#             minQueryLen = len(mfcc_feat)
            mfccPosnName[j] = filename.split('.')[0]
            j += 1
            i += stride
print(minQueryLen)
print(minId)
print(maxQueryLen)
print(maxId)
print(len(mfccNumpyArray))

61
<closed file '/home/ubuntu/spokenkeywordsearch/GroundTruth/QueryAudio/1007.wav', mode 'rb' at 0x7faa10260ae0>
169
<closed file '/home/ubuntu/spokenkeywordsearch/GroundTruth/QueryAudio/379.wav', mode 'rb' at 0x7faa10416150>
176225


In [25]:
print(mfccPosnName[0])

470


In [30]:
import numpy as np
import lshknn

# Make mock data
# 2 features (rows), 4 samples (columns)
res = mfccNumpyArray[0:5] + audioMfcc
data = np.array(res,
        dtype=np.float64)
data = data.transpose()

print(data.shape)
print(len(mfccNumpyArray))
print(len(mfccNumpyArray[0]))

# Instantiate class
c = lshknn.Lshknn(
        data=data,
        k=20,
        threshold=0.2,
        m=10,
        slice_length=4)

# Call subroutine
knn, similarity, n_neighbors = c()

# Check result
# print(knn)

(1300, 34642)
3226
1300


In [31]:
print(knn[0:5])

[[8845L 33938L 10386L 24740L 1148L 2655L 29603L 11316L 4239L 29509L
  11308L 33841L 5206L 25424L 23589L 25343L 1442L 5058L 19691L 509L]
 [7476L 9054L 2695L 26697L 23920L 21555L 13018L 3044L 12021L 13779L
  22372L 26857L 10710L 10688L 11876L 27846L 19015L 33384L 2197L 1535L]
 [27596L 14457L 23397L 3373L 14344L 22934L 23499L 22996L 14423L 14305L
  3339L 9961L 3499L 1205L 23872L 1211L 3683L 103L 414L 998L]
 [29992L 17393L 29681L 4442L 4610L 22414L 29519L 24987L 29447L 11336L
  28017L 13268L 17814L 33100L 3198L 10212L 22177L 3870L 947L 30107L]
 [8903L 11854L 2180L 31975L 31930L 6284L 11964L 14268L 2173L 4384L 1063L
  4463L 2219L 32519L 16536L 6011L 569L 102L 380L 32527L]]


In [32]:
print(set(knn[0]).intersection(set(knn[4])))

set([])


In [24]:
print(audioPosnName[1232-5])
print(audioPosnName[1298-5])
print(audioPosnName[1667-5])

2086-149220-0003
2086-149220-0000
2086-149220-0049


In [15]:
# print(queryList[470])
# # print(transDict['2086-149220-0003'])
# # print(transDict['2086-149220-0000'])
# # print(transDict['2086-149220-0049'])
# for i in knn[0:5]:
#     for j in i:
#         if(audioPosnName[j-5]=='1272-141231-0026'):
#             print("Got it")
#         else:
#             print(audioPosnName[j-5])

In [39]:
print((queryList[146]))
for q in encodeAudioDict:
    if(encodeAudioDict[q]=='1919-142785-0024.flac.json'):
        print(q)
# print(encodeQueryDict[3])
# print(type(encodeForQuery[0]))

bestia 
8938
8939
8940
8941
8942
8943
8944
8945


In [18]:
import numpy as np
import lshknn

# Make mock data
# 2 features (rows), 4 samples (columns)
res = encodeForQuery[1947:1953] + (encodeForAudio)
# print(len(res))
# print(len(res[0]))
data = np.array([np.array(x) for x in res])#, dtype=np.float64)
data = data.transpose()

print(data.shape)

# Instantiate class
c = lshknn.Lshknn(
        data=data,
        k=20,
        threshold=0.2,
        m=10,
        slice_length=4)

# Call subroutine
knn, similarity, n_neighbors = c()

# Check result
# print(knn)

(120, 68062)


In [19]:
print(knn[0:6])

[[52166L 18341L 57667L 57841L 58212L 4491L 4398L 31066L 51673L 51887L
  54901L 17835L 54654L 6049L 58551L 2L 4L 2923L 166L 1280L]
 [6218L 8450L 26997L 47154L 64527L 8435L 64530L 64553L 41592L 33591L
  32638L 3L 5L 25174L 2155L 1459L 1464L 2072L 1589L 1752L]
 [52166L 18341L 57667L 57841L 58212L 4491L 4398L 31066L 51673L 51887L
  54901L 17835L 54654L 6049L 58551L 0L 4L 2923L 166L 1280L]
 [6218L 8450L 26997L 47154L 64527L 8435L 64530L 64553L 41592L 33591L
  32638L 1L 5L 25174L 2155L 1459L 1464L 2072L 1589L 1752L]
 [52166L 18341L 57667L 57841L 58212L 4491L 4398L 31066L 51673L 51887L
  54901L 17835L 54654L 6049L 58551L 0L 2L 2923L 166L 1280L]
 [6218L 8450L 26997L 47154L 64527L 8435L 64530L 64553L 41592L 33591L
  32638L 1L 3L 25174L 2155L 1459L 1464L 2072L 1589L 1752L]]


In [42]:
for k in knn[8938+6:8945+6]:
    count = 0
    for c in k:
        count += 1
        if(c>=2):
#             print(encodeAudioDict[c-2])
#             print(transDict[encodeAudioDict[c-2].split('.')[0]])
            if('illustration' in transDict[encodeAudioDict[c-2].split('.')[0]]):
                print(transDict[encodeAudioDict[c-2].split('.')[0]])
                print(count)
                print('True')

illustration ginger

12
True


In [13]:
count = 0
for q in queryList:
    if(q == 'illustration '):
        print(count)
        break
    count += 1

49


In [36]:
print(knn[19813+6:19819+6])

[[2628L 3361L 3913L 4351L 4369L 4612L 4650L 3833L 2741L 2703L 2803L 2453L
  2031L 1853L 1805L 1783L 1349L 493L 483L 288L]
 [2372L 20205L 19716L 19197L 20246L 54281L 38723L 3459L 37395L 37424L
  61473L 22028L 60150L 22142L 60020L 26380L 49100L 58468L 1163L 2135L]
 [45547L 17090L 29077L 65732L 62279L 2575L 2571L 28896L 17027L 16951L
  64973L 53298L 53791L 6011L 457L 28352L 1304L 1571L 2017L 573L]
 [26587L 2457L 16650L 19452L 54400L 53906L 14318L 53855L 41629L 50000L
  9804L 23033L 39437L 16641L 18175L 25782L 55580L 7464L 58941L 49550L]
 [27037L 57822L 26173L 10149L 5472L 14286L 4216L 43902L 4967L 55457L
  12262L 4937L 55388L 14859L 17407L 11478L 8370L 59299L 4648L 12879L]
 [9188L 55090L 53480L 52374L 21589L 21625L 11964L 66424L 43692L 52722L
  361L 50363L 54018L 41540L 21390L 6197L 10917L 30344L 28141L 962L]]


In [ ]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(audioMfcc)

In [ ]:
distances, indices = nbrs.kneighbors(mfccNumpyArray)

In [ ]:
schema = Schema(audio=TEXT(stored=True), content=TEXT)
if not os.path.exists("index"):
    os.mkdir("index")
index = create_in("index", schema)
writer = index.writer()
for a in transDict:
    writer.add_document(audio=u""+a, content=u""+transDict[a])
writer.commit()
searcher = index.searcher()

In [104]:
log = open('SimpleMFCCKNNSearch', 'a')
commonCount = 0
totalCount = 0
maxMatching = 0
i = 0
while(i < (len(mfccPosnName))):
    samePos = []
    while(i+1<len(mfccPosnName) and mfccPosnName[i+1]==mfccPosnName[i]):
        samePos.append(i)
        i += 1
    totalCount += 1
    i -= 1
    log.write('------------------------------------------------------------\n')
    log.write(mfccPosnName[i] + '\n')
    log.write(queryList[int(mfccPosnName[i])] + '\n')
    query = QueryParser("content", index.schema, group=qparser.OrGroup).parse(queryList[int(mfccPosnName[i])])
    results = searcher.search(query, limit=10)
    list1 = []
    list2 = []
    for res in results:
        list1.append(res['audio'])
    for ind in samePos:
        for k in indices[ind]:
            if(audioPosnName[k]):
                log.write(audioPosnName[k]+" : ")
    #             print(str(audioPosnName[k]) + "--")
                log.write(transDict[str(audioPosnName[k])] + "\n")
                list2.append(audioPosnName[k])
    comm = len(set(list1).intersection(set(list2)))
    if(comm>maxMatching):
        maxMatching = comm
    commonCount += comm
    log.write("Matching : {0}".format(comm))
    log.flush()
    i += 1
print(commonCount)
print(totalCount)
print(maxMatching)
print(commonCount/float(totalCount))

892
6256
4
0.142583120205


In [88]:
if '6241-61946-0020' in transDict:
    transDict['84-121123-0000']

In [108]:
# minQueryLen = 100000
# featSize = 50
# # mfccDict = dict(list())
# audiofBank = []
# audiofPosnName = dict()
# count = 0
# j = 0
# for root, dirnames, filenames in os.walk('/home/ubuntu/DevData-AudioBooks/dev-clean/'):
#     for filename in fnmatch.filter(filenames, '*.wav'):
#         count += 1
#         with open(os.path.join(root, filename), 'rb') as f:
#             data, samplerate = sf.read(f)
# #         print("Read sounfile done")
#         fbank_feat = np.array(fbank(data, samplerate))
# #         print("Mfcc done")
#         mfcclen = len(mfcc_feat)
#         i = 0
#         while(i+featSize<mfcclen):
#             audiofBank.append(fbank_feat[i:i+featSize].flatten())
# #         if(len(mfcc_feat)<minQueryLen):
# #             minQueryLen = len(mfcc_feat)
#             audiofPosnName[j] = filename.split('.')[0]
#             i += 25
#             j += 1
# print(len(audiofBank))
# print(mfcclen)
# print(count)